<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>Basado en: &copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados. Modificado el 2018-1.</font>
</p>

## _Properties_

En los lenguajes OOP, el encapsulamiento suele proveerse mediantes atributos _públicos_ o _privados_. En Python, como ya hemos mencionado, **todos** los atributos y métodos de una clase son **públicos**, y el hecho de iniciar el nombre de un atributo o método con _underscore_ es una convención (y una buena práctica), pero no asegura un caracter privado de estos elementos.

La manera que tiene Python de proveer encapsulamiento es mediante el uso de un mecanismo llamado _properties_.

### ¿Qué son las _properties_?

En Python, una _property_ funciona como un atributo, pero sobre el cual podemos modificar su comportamiento cada vez que es leído (`get`), escrito (`set`), o eliminado (`del`). Al usar el mecanismo de _properties_ sobre un atributo, podemos ejecutar acciones de manera más limpia que invocando métodos explícitos para leer o modificar el valor de un objeto.

### ¿Para qué las _properties_?

Tomemos un ejemplo sencillo en que deseamos definir una clase que modela un puente a través del cual puede pasar solo una cantidad máxima de personas por motivos de seguridad. Una manera inicial de implementarlo es mediante la siguiente clase.


In [1]:
class Puente:
    
    def __init__(self, maximo):
        self.maximo = maximo
        self.personas = 0
        
puente = Puente(10)
puente.personas += 7
print(f"Hay {puente.personas} personas en el puente.")
puente.personas += 5
print(f"Hay {puente.personas} personas en el puente.")
puente.personas -= 15
print(f"Hay {puente.personas} personas en el puente.")

Hay 7 personas en el puente.
Hay 12 personas en el puente.
Hay -3 personas en el puente.


En este ejemplo es fácil modificar la cantidad de personas en el puente de manera externa a él, simplemente accediendo al atributo (público) `personas`. Sin embargo, al permitir estas modificaciones, también permitimos que el puente quede con más personas de las permitidas, o bien con una cantidad negativa de personas. La manera obvia de corregir esto es agregando acciones al momento de modificar la cantidad de personas.

In [2]:
class Puente:
    
    def __init__(self, maximo):
        self.maximo = maximo
        self.personas = 0
        
puente = Puente(10)
puente.personas += 7
if puente.personas > puente.maximo:
    puente.personas = puente.maximo
print(f"Hay {puente.personas} personas en el puente.")

puente.personas += 5
if puente.personas > puente.maximo:
    puente.personas = puente.maximo
print(f"Hay {puente.personas} personas en el puente.")

puente.personas -= 15
if puente.personas < 0:
    puente.personas = 0
print(f"Hay {puente.personas} personas en el puente.")

Hay 7 personas en el puente.
Hay 10 personas en el puente.
Hay 0 personas en el puente.


Este código permite que el puente funcione de manera correcta. Sin embargo, el código ahora es más complicado de **leer** y de **mantener**. Cada vez que modificamos el valor del atributo `personas` _debemos recordar_ efectuar ciertas verificaciones que (1) entorpecen el flujo natural del programa y que (2) violan el principio de encapsulamiento ya que estas verificaciones deberían ser responsabilidad de la clase `Puente`. La siguiente mejora agrega las verificaciones dentro de la clase, y **encapsula** el atributo `persona`.

In [3]:
class Puente:
    
    def __init__(self, maximo):
        self.maximo = maximo
        self.__personas = 0
        
    def contar(self):
        return self.__personas
    
    def ingresar(self, p):
        if self.__personas + p > self.maximo:
            self.__personas = self.maximo
        elif self.__personas + p < 0:
            self.__personas = 0
        else:
            self.__personas += p
            
            
puente = Puente(10)
puente.ingresar(7)
print(f"Hay {puente.contar()} personas en el puente.")
puente.ingresar(5)
print(f"Hay {puente.contar()} personas en el puente.")
puente.ingresar(-15)
print(f"Hay {puente.contar()} personas en el puente.")

Hay 7 personas en el puente.
Hay 10 personas en el puente.
Hay 0 personas en el puente.


Hemos movido el comportamiento que verifica las condiciones a métodos de la clase `Puente`. Hemos encapsulado el acceso y modificación del atributo `__personas` dentro de los métodos `contar` e `ingresar` quienes tienen, respectivamente, la misión de leer u obtener (método _getter_) y modificar o actualizar (método _setter_) el valor del atributo interno `__personas`. Nuestro código es más fácil de leer. 

Algo que podemos lamentar es que, dentro de todo, la primera versión que leía y modificaba directamente el atributo `personas` tenía una sintaxis más sencilla y ahora, en cambio, debemos llamar métodos particulares `ingresar` y `contar`. Más aún, en el futuro, si decidimos cambiar los nombres de los métodos `ingresar` y `contar`, debemos buscar todas las veces en que los hemos usado fuera de la clase y modificarlos.

La mejor combinación de ambos mundos: encapsulamiento, y sintaxis más simple, la provee el mecanismo de **properties**. Para incorporarlo veamos el siguiente ejemplo:

In [6]:
class Puente:
    
    def __init__(self, maximo):
        self.maximo = maximo
        self.__personas = 0
        
    @property
    def asdf(self): # reemplaze personas por asdf para ver que property no necesita relacion con atributo a manejar
        return self.__personas

    @asdf.setter
    def asdf(self, p):
        if p > self.maximo:
            self.__personas = self.maximo
        elif p < 0:
            self.__personas = 0
        else:
            self.__personas = p
            
            
puente = Puente(10)
puente.asdf += 7
print(f"Hay {puente.asdf} personas en el puente.")
puente.asdf += 5
print(f"Hay {puente.asdf} personas en el puente.")
puente.asdf -= 15
print(f"Hay {puente.asdf} personas en el puente.")

Hay 7 personas en el puente.
Hay 10 personas en el puente.
Hay 0 personas en el puente.


Al escribir el decorador `@property` antes del método `personas`, estamos definiendo una _property_ de nombre `personas`. Esta _property_ se comporta como un atributo cuyo método _getter_ es precisamente el método `personas`. Adicionalmente podemos definir otro método como _setter_ (que nos permitirá modificar el valor de la _property_) y para eso lo decoramos con `@personas.setter`.

El código resultante que escribimos fuera de la clase es tan simple como en la primera versión, y además encapsula, dentro de los métodos _getter_ y _setter_ para `personas`, el comportamiento que verifica que las restricciones sobre el puente se cumplan.

### Otras maneras de definir _properties_

Si no le parece tan clara la manera definir _properties_ mediante decoradores, Python ofrece otra manera más explícita que provee el mismo comportamiento. Observar la siguiente versión de la clase `Puente`:

In [5]:
class Puente:
    
    def __init__(self, maximo):
        self.maximo = maximo
        self.__personas = 0
        
    def _get_personas(self):
        return self.__personas

    def _set_personas(self, p):
        if p > self.maximo:
            self.__personas = self.maximo
        elif p < 0:
            self.__personas = 0
        else:
            self.__personas = p
        
    personas = property(_get_personas, _set_personas)

    
puente = Puente(10)
puente.personas += 7
print(f"Hay {puente.personas} personas en el puente.")
puente.personas += 5
print(f"Hay {puente.personas} personas en el puente.")
puente.personas -= 15
print(f"Hay {puente.personas} personas en el puente.")

Hay 7 personas en el puente.
Hay 10 personas en el puente.
Hay 0 personas en el puente.


Esta vez hemos definido los métodos "pseudo-privados" `_get_personas` y `_set_personas`, y posteriormente hemos definido un atributo `personas` dentro de la clase, pero fuera de los otros métodos (_atributo de clase_). Este atributo se define como una [`property`](https://docs.python.org/3/library/functions.html#property) y se le indica que sus métodos _getter_ y _setter_ serán, respectivamente, `_get_personas` y `_set_personas`. 

El comportamiento es el mismo que en el caso anterior que usaba decoradores. Las verificaciones se encuentran encapsuladas en los métodos _getter_ y _setter_ asociados a la _property_ `personas`; la sintaxis es simple y no depende de los nombres internos de los métodos _getter_ y _setter_.

### Ejemplo: caching de páginas web

Un típico ejemplo de acción invocada es cuando hacemos _caching_ de una página web. Esto ocurre cuando nuestro navegador guarda contenido del sitio, para no tener que descargarlo cada vez que se accede a él. 

En nuestro ejemplo, un atributo que corresponde al contenido de una página web. Si un usuario accede al contenido por primera vez, descargamos el contenido y lo guardarmos. De esta forma, en los próximos accesos podemos retornar el contenido guardado sin la necesidad de bajarlo de nuevo.

In [6]:
from urllib.request import urlopen


class WebPage:

    def __init__(self, url):
        self.url = url
        self._content = None
        
    @property
    def content(self):
        if not self._content:
            print("Obteniendo página web...")
            self._content = urlopen(self.url).read()
        return self._content

In [7]:
import time

page = WebPage("http://www.puc.cl")
now = time.time()  # devuelve el tiempo en segundos
contenido_1 = page.content
print(f"Tiempo en obtener la página por primera vez: {time.time() - now} segundos.")

now = time.time()
contenido_2 = page.content
print(f"Tiempo en obtener la página por segunda vez: {time.time() - now} segundos.")

contenido_1 == contenido_2

Obteniendo página web...
Tiempo en obtener la página por primera vez: 1.2363169193267822 segundos.
Tiempo en obtener la página por segunda vez: 0.0001678466796875 segundos.


True

### Ejemplo: definiendo una clase `Email`

Una forma de usar properties es definiendo los métodos y luego asignarlos a una variable usando `property`.

In [8]:
class Email:
    
    def __init__(self, address):
        self._email = address
        
    def _get_email(self):
        return self._email
        
    def _set_email(self, value):
        if '@' not in value:
            print("Esto no parece una dirección de correo.")
        else:
            self._email = value
    
    def _del_email(self):
        print("¡Eliminaste el correo!")
        del self._email    

    email = property(_get_email, _set_email, _del_email, "Esta propiedad corresponde al correo.")

In [9]:
help(Email)

Help on class Email in module __main__:

class Email(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, address)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  email
 |      Esta propiedad corresponde al correo.



In [10]:
mail = Email("kp1@gmail.com")
print(mail.email)
mail.email = "kp2@gmail.com"
print(mail.email)
mail.email = "kp2.com"
del mail.email

kp1@gmail.com
kp2@gmail.com
Esto no parece una dirección de correo.
¡Eliminaste el correo!


Ojo que el código no nos prohibe acceder directamente al atributo `_email` saltándonos el método `_set_email`:

In [11]:
mail._email = "kp3.com"
mail._email

'kp3.com'

La _property_ es simplemente una referencia al mismo atributo `_email`.

Para probar esto podemos utilizar el operador `is`. El operador `is` es un _test_ para la identidad de un objeto. `x is y` es verdadero si y sólo si `x` e `y` son el mismo objeto.

In [12]:
mail.email is mail._email

True

La forma más típica de usar _properties_ es usar los decoradores. Veremos decoradores varias clases más adelante. Como ejemplo reutilizaremos la clase `Email`

In [13]:
class Email2:
    
    def __init__(self, address):
        self._email = address
    
    @property
    def email(self):
        return self._email
    
    @email.setter
    def email(self, value):
        if '@' not in value:
            print("Esto no parece una dirección de correo.")
        else:
            self._email = value

    @email.deleter
    def email(self):
        print("¡Eliminaste el correo!")
        del self._email

Se puede observar que el funcionamiento de `Email2` es equivalente al de `Email`:

In [14]:
mail = Email2("kp1@gmail.com")
print(mail.email)
mail.email = "kp2@gmail.com"
print(mail.email)
mail.email = "kp2.com"
del mail.email

kp1@gmail.com
kp2@gmail.com
Esto no parece una dirección de correo.
¡Eliminaste el correo!


Las _properties_ con decoradores también pueden involucar acciones que dependen de variables de la clase:

In [7]:
class Email3:
    
    def __init__(self, address):
        self._email = address

    @property
    def username(self):
        return self._email.split('@')[0]

In [8]:
mail = Email3("kp1@gmail.com")
print(mail.username)
mail._email = "kp2@gmail.com"
print(mail.username)

kp1
kp2
